# Sentence Analysis
This workbook is to be employeed during analysis of the text

Set the following:
- connection info and 
- query to import data.

In [1]:
# Import moduls and 
# change directory to make it relative to top level moduls
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [2]:
# Import modules to be used in this package
import pandas as pd
from utils.lipht_data import getEngine

In [3]:
# Set the connection to the server
engine = getEngine('LIPHT-VM-01','Akademikernes_MSCRM_Addition')
query="""
        SELECT [ThreadID]
                        ,[ThreadTopic]
                        ,[SignalMessageID]
                        ,[SignalSource] AS ThreadInitiatedBy
                        ,[SignalMessageBodyClean]
                        ,[IsSystemGenerated]
                    FROM [Akademikernes_MSCRM_Addition].[out].[vw_LDA_MessagesALL]
    """

In [4]:
# Import data
df_scope = pd.read_sql(query, engine).copy(deep=True)

In the next section work has been done in order to do sentence tagging based on conventional danish way of formulating messages. This should be adapted into the scope of the project that you are working on.

In [5]:
import re
from utils.lipht_regex import re_accronyms, re_system, re_keyconcepts, re_lemma, re_remove

sentence_cleanup = {
    r'\xa0\n': '\n',
    r'\xa0': ' ',
    r'(\r\n)': ' ',
    r'(?=[\?])\?+(\s)?(?<=[\?])': '?\n',
    r'\sd\.': 'den',
    r'(?<=den\s\d{2})\.(?=(,|\s))|(?<=den\s\d{1})\.(?=(,|\s))|(?<=senest\s\d{2})\.(?=(,|\s))|(?<=senest\s\d{1})\.(?=(,|\s))': '', # replace '.' after den 23. or senest 23.
    r'(?<=\s\d{2})\.|(?<=\s\d{1})\.|(?<=\s\d{2}\.\d{2})\.|(?<=\s\d{1}\.\d{2})\.': '-', # handles 23.6 and 1.6 to '-' instead of '.'
    r'(?<=(den|til)\s\d{2})\.': '-', # swap 27.06 to 27-06
}

def blobtolist(blob):  
    list_sent = [s.strip() for s in re.split(r'\n', blob)] # Create a list by every linebreak
    list_sent = list(filter(None,list_sent)) # Remove all empty lists
    list_sent = [s.strip('.') for s in list_sent] # Remove '.' from sentences in each list

    final = []
    for i in list_sent:
        temp = re.split(r'\.',i)
        for i in temp:
            final.append(i.strip())
    return final

def tag_sentences(list_sentences):
    def find_matches(item):
        tags = {
            r'.*\?': 'QUESTION',
            r'(til|til\s*\svedkom|kære|hej|hejsa|goddag|godmorgen|hello|hi|dear)': 'GREETING',
            r'.*(bekræfter.*.hermed.*.korrekt)': 'CONFIRMATION',
            r'\b(tak)': 'POSITIVE FEEDBACK',
            r'.*(håber.*.hjælpe)': 'REQUEST HELP',
            r'.*(kan i tjekke.*\?)': 'REQUEST X CHECKED',
            r'.*(har i mulighed.*\?)': 'REQUEST FOR ACTION',
            r'.*(vedhæft(ede|et|er|e)?|fil(type(r)|er))': 'ATTACHMENT',
            r'([0-9]+\s){4}|(([0-9]){8})': 'PHONE NUMBER',
            r'(på\sforhånd\stak)|(((jeg|du)\sønske(s|r\s)(dig)?|(hav))\s)?(en\s)?((fortsat|rigtig)\s)?god(t)?\s(weekend|dag|ferie|jul|nytår|påske)|(held\sog\slykke\s(fremover)?)|(god\sarbejdslyst)': 'POLITE ENDING',
            r'(med\svenlig\shilsen|de\sbedste\shilsner|bedste\shilsener|bedste\shilsner|kærlig\shilsen|dbh|venlig\shilsen|mvh|m\.v\.h\.|vh|hilsen|venlig|bh|best\sregards|with\skind\sregards|kind\sregards|with\sregards|regards|kh).*': 'ENDING',
            r'(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])?': 'URL',
            r'([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])?': 'URL',
            r'(?<=\s)(\w)*@{1}(\w)*\.(\w){2,3}(?=\s)': 'EMAIL',
            r'\b(ak(\s)?(\d{3}))': 'AK FORM',
            r'\b(ar(\s)?(\d{3}))': 'AR FORM',
            r'(sent\sfrom\smy\s).*':'DEVICE INFORMATION',
            }
        for k in tags:
            if re.match(k, item):
                return tags[k]

    matches = []
    used_tags = []
    for sentence in list_sentences:
        sentence = sentence.lower().strip()
        if 'ENDING' in used_tags:
            # CREATE CHECK FOR things after ENDING
            matches.append((sentence,'AFTER ENDING'))
        elif find_matches(sentence) is None:
            matches.append((sentence,'CONTENT'))
        else:
            used_tags.append(find_matches(sentence))
            matches.append((sentence, find_matches(sentence)))
            next
    return matches


In the next section we are extracing only the text column to be used for further analysis. This will be followed by applying the functions from above on the target_col.

In [6]:
target_col = 'SignalMessageBodyClean'#'FirstMemberMessageBody'#'FirstResponseFromAKAToMemberMessageBody'

In [7]:
df_sentences = df_scope[[target_col]].copy(deep=True)

In [8]:
%%time
# df_sentences['clean_blob'] = df_sentences[target_col].apply(clean_blob) # Make list of strings
df_sentences['clean_blob'] = df_sentences[target_col].str.lower()
df_sentences['clean_blob'] = df_sentences['clean_blob'].replace(regex=re_accronyms) # Make list of strings
df_sentences['clean_blob'] = df_sentences['clean_blob'].replace(regex=sentence_cleanup) # Make list of strings
df_sentences['sentence_list'] = df_sentences['clean_blob'].apply(blobtolist) # Make list of strings
df_sentences['tagged_list'] = df_sentences['sentence_list'].apply(tag_sentences) # Tag each string in list -> tuples
df_sentences['list_of_tags'] = df_sentences['tagged_list'].apply(lambda df: [(t[1], t[0]) for t in df]) # swap order of k, v

Wall time: 14min 33s


See a sample of the what was produced by our fuctions - 4 new columns. 
- clean_blob - text preperation 
- sentence_list - the text split into sentences
- tagged_list - the text split into tuples of sentences and their tag
- list_of_tags - same as above, only swapped tupples

Run multiple times to see new samples (hint use: CTRL+ENTER)

In [9]:
# Sample of function results
sample = df_sentences.sample(1).index 
print('\nSample')
print([i for i in df_sentences['sentence_list'][sample]])
print('\nSample - with sentence tagging')
print([i for i in df_sentences['list_of_tags'][sample]])
print('\nSample - cleaned text')
print(''.join(df_sentences['clean_blob'][sample]))


Sample
[['du har optjent feriedagpenge, som du kan søge om til ferieåret 2018/2019', 'læs nærmere i brevet, hvor mange dage med feriedagpenge du har optjent, og hvordan du søger om dem']]

Sample - with sentence tagging
[[('CONTENT', 'du har optjent feriedagpenge, som du kan søge om til ferieåret 2018/2019'), ('CONTENT', 'læs nærmere i brevet, hvor mange dage med feriedagpenge du har optjent, og hvordan du søger om dem')]]

Sample - cleaned text
du har optjent feriedagpenge, som du kan søge om til ferieåret 2018/2019. læs nærmere i brevet, hvor mange dage med feriedagpenge du har optjent, og hvordan du søger om dem.


In [10]:
df_count = df_sentences['list_of_tags'] # Take the single column
df_list = df_count.tolist() # Convert dataframe to list of lists with tuples
tag_sentence_pair = [item for sublist in df_list for item in sublist] # convert to list of tuples

In [11]:
# Create dataframe from tag_sentence_pair
headers = ['tag', 'sentence']
df = pd.DataFrame(tag_sentence_pair, columns=headers)
df_sum = df.groupby(['sentence','tag']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)

In [12]:
%%time
# df_sum = df_sum[df_sum['counts']>1]
df_sum['column'] = target_col

Wall time: 13 ms


In [13]:
df_sum.sample(1)

,sentence,tag,counts,column
1611976,wohlsch@gmail,AFTER ENDING,1,SignalMessageBodyClean


In [14]:
df_sum[df_sum['tag']=='REQUEST HELP'].head(15).sort_values(by=['sentence'])

,sentence,tag,counts,column
671277,håber at i kan hjælpe,REQUEST HELP,18,SignalMessageBodyClean
671283,håber at i kan hjælpe mig,REQUEST HELP,15,SignalMessageBodyClean
672460,håber det kunne hjælpe,REQUEST HELP,74,SignalMessageBodyClean
673477,håber du kan hjælpe,REQUEST HELP,31,SignalMessageBodyClean
673490,håber du kan hjælpe mig,REQUEST HELP,17,SignalMessageBodyClean
673918,håber i kan hjælpe,REQUEST HELP,304,SignalMessageBodyClean
673927,håber i kan hjælpe :),REQUEST HELP,17,SignalMessageBodyClean
674023,håber i kan hjælpe mig,REQUEST HELP,157,SignalMessageBodyClean
958478,jeg håber at i kan hjælpe,REQUEST HELP,15,SignalMessageBodyClean
961414,jeg håber i kan hjælpe,REQUEST HELP,80,SignalMessageBodyClean


In [15]:
df_sum.shape

(1633966, 4)

In [16]:
%%time
df_sum.to_sql(name='sentence_count'.format(target_col) ,con=engine , schema='input', if_exists='replace', index=False)


Wall time: 4min 34s


In [17]:
'Done'

'Done'

In [ ]:
sample = df_sentences['FirstMemberMessageBody'].sample(1).index[0]
test = df_sentences['FirstMemberMessageBody'][sample]
l = tag_sentences(blobtolist(test))
test, l

In [ ]:
l = [(t[1], t[0]) for t in l] # swap tuple elements
l

In [ ]:
table = [[t[1], t[0]] for t in l]
headers = ['sentence', 'tag']
df = pd.DataFrame(table, columns=headers)
df

In [ ]:
d = {}
for k, v in l:
    d[k] = d.get(k, ()) + (v,)
d

In [ ]:
df = pd.DataFrame(d, index['i'])
df

In [ ]:
import re
test = re.sub('(vedhæft(ede|et|er|e)?)','snakket','jeg har vedhæfte følgende dokument')
test

In [ ]:
re.sub(r'\b(hold(et|er|et|e|t)?)','holde','Både holdet, hold og holder skulle skiftes') #r'\b(holdet|holder|holde|hold)